# Strategy Serialization with BoFire

## Imports

In [1]:
import json
from pydantic import parse_obj_as, BaseModel, Field
from typing import List, Dict, Optional, Literal


from bofire.data_models.domain.api import Inputs, Outputs, Domain
from bofire.benchmarks.single import Himmelblau
from bofire.benchmarks.multi import DTLZ2
from bofire.data_models.strategies.api import SoboStrategy as SoboStrategyDataModel
from bofire.data_models.strategies.api import QnehviStrategy as QnehviStrategyDataModel
from bofire.data_models.strategies.api import RandomStrategy as RandomStrategyDataModel
from bofire.data_models.strategies.api import AnyStrategy
from bofire.data_models.acquisition_functions.api import qNEI
import bofire.strategies.api as stategies
from bofire.data_models.surrogates.api import BotorchSurrogates, AnySurrogate, SingleTaskGPSurrogate
from bofire.data_models.kernels.api import ScaleKernel, RBFKernel
from bofire.surrogates.trainable import TrainableSurrogate
from bofire.surrogates.diagnostics import CvResults2CrossValidationValues, CrossValidationValues




/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Single Objective Problem Setup

In [2]:
benchmark = Himmelblau()
samples = benchmark.domain.inputs.sample(n=10)

# this is the training data
experiments = benchmark.f(samples, return_complete=True)

# this are the pending candidates
pending_candidates = benchmark.domain.inputs.sample(2)



## Random Strategy

The random strategy and other strategies that just inherit from `Strategy` and not `PredictiveStrategy` are special as they do not need defined output features in the domain and they do not need a call to `tell` before the `ask`. Furthermore they online provide input features in the candidates and no predictions for output features.

In [3]:
# setup the data model
domain = Domain(input_features=benchmark.domain.input_features)
strategy_data = RandomStrategyDataModel(domain=domain)

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "RandomStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}, "output_features": {"type": "Outputs", "features": []}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": 632}'

In [4]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# ask it
df_candidates = strategy.ask(candidate_count=5)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value=-0.39832076260392757), 'x_2': InputValue(value=-0.7609607369925802)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=0.5497314129320796), 'x_2': InputValue(value=-5.046732447716293)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=1.3452722401899244), 'x_2': InputValue(value=1.597640790269045)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=-3.9037743321454013), 'x_2': InputValue(value=0.3398091702288455)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=4.6989467277800205), 'x_2': InputValue(value=-1.7212722298533234)}, outputValues=None)]

## SOBO Strategy

This will fail as SOBO is a predictive strategy which needs also output feature definitions, which is missing in the domain from before.

In [5]:
# setup the data model
strategy_data = SoboStrategyDataModel(domain=domain, acquisition_function=qNEI())

# we generate the json spec
jspec = strategy_data.json()

jspec

ValidationError: 1 validation error for SoboStrategy
domain
  no output feature specified (type=value_error)

Next try with a correct domain:

In [6]:
# setup the data model
strategy_data = SoboStrategyDataModel(domain=benchmark.domain, acquisition_function=qNEI())

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "SoboStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}, "output_features": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}]}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": 253, "num_sobol_samples": 512, "num_restarts": 8, "num_raw_samples": 1024, "descriptor_method": "EXHAUSTIVE", "categorical_method": "EXHAUSTIVE", "discrete_method": "EXHAUSTIVE", "surrogate_specs": {"surrogates": [{"type": "SingleTaskGPSurrogate", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0]}]}, "outp

This will fail as SOBO is a predictive strategy which means we have to provide training data before:

In [7]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# ask it
df_candidates = strategy.ask(candidate_count=2)

ValueError: Not enough experiments available to execute the strategy.

which is done by using the `tell` method:

In [9]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# tell it the pending candidates if present
if pending_candidates is not None:
    strategy.add_candidates(pending_candidates)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=2)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value=-1.2012287772870565), 'x_2': InputValue(value=2.3839681325410282)}, outputValues={'y': OutputValue(predictedValue=-27.03056758938328, standardDeviation=169.00815870878822, objective=27.03056758938328)}),
 Candidate(inputValues={'x_1': InputValue(value=5.58215853323273), 'x_2': InputValue(value=-0.6109173766848464)}, outputValues={'y': OutputValue(predictedValue=43.317914949450596, standardDeviation=242.31451628448355, objective=-43.317914949450596)})]

We can also save the trained models of the strategy, for more info look at the `model_serial.ipynb` notebook. It could be that the `dumps` command fails here. But this is already fixed in the main branch of the `linear_operator` package, and if not yet, it should be available in main soon.

In [10]:
jsurrogate_spec = strategy_data.surrogate_specs.surrogates[0].json()
dump = strategy.surrogate_specs.surrogates[0].dumps()

## MOBO Strategy

As example for a multiobjective strategy we are using here the Qnehvi stratey. Related strategies would be Qparego, MultiplicativeSobo etc. To use it, we have to first generate a multiobjective domain.

In [11]:
benchmark = DTLZ2(dim=6)
samples = benchmark.domain.inputs.sample(n=20)
experiments = benchmark.f(samples, return_complete=True)
pending_candidates = benchmark.domain.inputs.sample(2)


Now the strategy spec is setup. Note that we can define there exactly which model to use.

In [12]:
# setup the data model
strategy_data = QnehviStrategyDataModel(
    domain=benchmark.domain,
    surrogate_specs=BotorchSurrogates(
        surrogates=[
            SingleTaskGPSurrogate(
                input_features=benchmark.domain.input_features,
                output_features=Outputs(features=[benchmark.domain.outputs[0]]),
                kernel=ScaleKernel(base_kernel=RBFKernel(ard=False))
            )
        ]
    )
)

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "QnehviStrategy", "domain": {"type": "Domain", "input_features": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_0", "unit": null, "bounds": [0.0, 1.0]}, {"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [0.0, 1.0]}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [0.0, 1.0]}, {"type": "ContinuousInput", "key": "x_3", "unit": null, "bounds": [0.0, 1.0]}, {"type": "ContinuousInput", "key": "x_4", "unit": null, "bounds": [0.0, 1.0]}, {"type": "ContinuousInput", "key": "x_5", "unit": null, "bounds": [0.0, 1.0]}]}, "output_features": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "f_0", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}, {"type": "ContinuousOutput", "key": "f_1", "objective": {"type": "MinimizeObjective", "w": 1.0, "lower_bound": 0, "upper_bound": 1}, "unit": null}]}, "constraints": {"type": "Constraints", "constraints": []}}, "seed":

Generate the candidates.

In [14]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# tell it the pending candidates if available
if pending_candidates is not None:
    strategy.add_candidates(pending_candidates)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=1)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_0': InputValue(value=1.0), 'x_1': InputValue(value=0.0), 'x_2': InputValue(value=1.0), 'x_3': InputValue(value=0.208562612543878), 'x_4': InputValue(value=0.0), 'x_5': InputValue(value=0.2329520868935282)}, outputValues={'f_0': OutputValue(predictedValue=-0.1475980614994069, standardDeviation=0.24772132745479597, objective=0.1475980614994069), 'f_1': OutputValue(predictedValue=1.1182802266356866, standardDeviation=0.3141427969914292, objective=-1.1182802266356866)})]

To fill the model info section accordingly, the following snippet has to be executed for every surrogate, incldung saving the actual models.

In [15]:
class TestMethod(BaseModel):
    type: str

class CrossValidation(TestMethod):
    type: Literal["CrossValidation"] = "CrossValidation"
    foldCount: int


for i in range(len(strategy_data.surrogate_specs.surrogates)):
    surrogate_data = strategy_data.surrogate_specs.surrogates[i]
    surrogate = strategy.surrogate_specs.surrogates[i]
    # get the spec
    jsurrogate_spec = surrogate_data.json()
    # get the dump
    dump = surrogate.dumps()
    # do the cross validation, only if we have a trainable model under the hood
    if isinstance(surrogate,TrainableSurrogate):
        cv_train, cv_test, _ = surrogate.cross_validate(strategy.experiments, folds=5)
        # transform the bofire objects to the backend objects
        testMethod = CrossValidation(foldCount=5)
        cvResultsTrain = CvResults2CrossValidationValues(cv_train)
        cvResultsTest = CvResults2CrossValidationValues(cv_test)
        metricsTrain = cv_train.get_metrics(combine_folds=False).describe().loc["mean"].to_dict()
        metricsTest = cv_test.get_metrics(combine_folds=True).describe().loc["mean"].to_dict()
        # save to backend
        # - jsurrogate_spec
        # - dump
        # - testMethod
        # - cvResultsTrain
        # - cvResultsTest
        # - metricsTrain
        # - metricsTest